In [15]:
from PIL import Image, ImageDraw, ImageFont
import os
import re

class ImageTextGen:
    def __init__(self, canvas_size=(128, 128), font_path=None, font_size=10, custom_space_width=None):
        """
        Inizializza il generatore.

        Args:
            canvas_size (tuple): (width, height).
            font_path (str): Path al file .ttf.
            font_size (int): Dimensione font.
            custom_space_width (int, optional):
                - Se None: usa spaziatura standard del font.
                - Se int: usa spaziatura custom per gli spazi (avanzamento fisso).
        """
        self.width, self.height = canvas_size
        self.font_size = font_size
        self.custom_space_width = custom_space_width

        if font_path and os.path.exists(font_path):
            self.font = ImageFont.truetype(font_path, font_size)
        else:
            print(f"Warning: Font '{font_path}' non trovato. Uso default.")
            self.font = ImageFont.load_default()

    def _get_line_height(self):
        """Calcola altezza riga in modo sicuro per entrambi i metodi."""
        bbox = self.font.getbbox("Hg")
        return bbox[3] - bbox[1] + 1

    def render(self, text):
        """Metodo principale che smista la logica."""
        if self.custom_space_width is not None:
            return self._render_custom(text)
        else:
            return self._render_standard(text)

    # ----------------------------
    # Helpers: char-level fitting
    # ----------------------------
    def _fit_prefix_len(self, s: str, max_w: float, measure_fn) -> int:
        """
        Ritorna la lunghezza massima del prefisso di `s` che entra in `max_w`.
        Usa binary search su measure_fn(s[:k]).
        """
        if not s:
            return 0

        eps = 1e-6
        # Se non entra nemmeno 1 char, forziamo 1 per evitare loop infinito
        if measure_fn(s[:1]) > max_w + eps:
            return 1

        lo, hi = 1, len(s)
        while lo < hi:
            mid = (lo + hi + 1) // 2
            if measure_fn(s[:mid]) <= max_w + eps:
                lo = mid
            else:
                hi = mid - 1
        return lo

    # ==========================================
    # LOGICA STANDARD (Veloce, Spaziatura Font)
    # ==========================================
    def _render_standard(self, text):
        lines = self._wrap_text_standard_charwise(text)
        line_height = self._get_line_height()

        images = []
        current_image = Image.new('L', (self.width, self.height), color=0)
        draw = ImageDraw.Draw(current_image)
        y_cursor = 0

        for line in lines:
            if y_cursor + line_height > self.height:
                images.append(current_image)
                current_image = Image.new('L', (self.width, self.height), color=0)
                draw = ImageDraw.Draw(current_image)
                y_cursor = 0

            # line può essere "" (riga vuota): ok, avanza comunque
            if line:
                draw.text((0, y_cursor), line, font=self.font, fill=255)
            y_cursor += line_height

        images.append(current_image)
        return images

    def _wrap_text_standard_charwise(self, text: str):
        """
        Wrap a livello carattere usando la misura reale del font.
        Mantiene i newline espliciti.
        """
        # Normalizza newline
        text = text.replace("\r\n", "\n").replace("\r", "\n")

        lines = []
        for paragraph in text.split("\n"):
            # Preserva righe vuote
            if paragraph == "":
                lines.append("")
                continue

            remaining = paragraph
            while remaining:
                if self.font.getlength(remaining) <= self.width:
                    lines.append(remaining.rstrip())
                    break

                k = self._fit_prefix_len(remaining, self.width, self.font.getlength)
                chunk = remaining[:k].rstrip()
                lines.append(chunk)

                remaining = remaining[k:]
                # Evita che le righe wrappate inizino con spazi (tipico comportamento)
                remaining = remaining.lstrip(" ")

        return lines

    # ==========================================
    # LOGICA CUSTOM (Precisa, Spaziatura Manuale)
    # ==========================================
    def _render_custom(self, text):
        """
        Render custom: spazi con avanzamento fisso (custom_space_width),
        testo misurato e disegnato in run (per performance/kerning interno alla run).
        """
        lines_tokens = self._wrap_text_custom_charwise(text)
        line_height = self._get_line_height()
        space_w = float(self.custom_space_width)

        images = []
        current_image = Image.new('L', (self.width, self.height), color=0)
        draw = ImageDraw.Draw(current_image)
        y_cursor = 0

        for tokens in lines_tokens:
            if y_cursor + line_height > self.height:
                images.append(current_image)
                current_image = Image.new('L', (self.width, self.height), color=0)
                draw = ImageDraw.Draw(current_image)
                y_cursor = 0

            x_cursor = 0.0
            for tok in tokens:
                if tok == " ":
                    x_cursor += space_w
                else:
                    draw.text((x_cursor, y_cursor), tok, font=self.font, fill=255)
                    x_cursor += self.font.getlength(tok)

            y_cursor += line_height

        images.append(current_image)
        return images

    def _wrap_text_custom_charwise(self, text: str):
        """
        Wrap a livello carattere con gestione spazi a larghezza fissa.
        - Preserva newline espliciti
        - Preserva spazi multipli (ma non li mette a inizio riga dopo wrap)
        - Spezza anche parole lunghissime a livello char
        """
        text = text.replace("\r\n", "\n").replace("\r", "\n")
        # Tratta i tab come 4 spazi (puoi cambiare se ti serve)
        text = text.replace("\t", "    ")

        space_w = float(self.custom_space_width)
        lines = []

        for paragraph in text.split("\n"):
            if paragraph == "":
                lines.append([])  # riga vuota
                continue

            # Runs: o sequenze di spazi, o sequenze di non-spazi
            runs = re.findall(r" +|[^ ]+", paragraph)

            current_tokens = []
            current_w = 0.0

            def flush_line():
                nonlocal current_tokens, current_w
                lines.append(current_tokens)
                current_tokens = []
                current_w = 0.0

            for run in runs:
                if not run:
                    continue

                if run[0] == " ":
                    # Spazi: aggiungi char per char (larghezza fissa)
                    for _ in run:
                        # Evita spazi a inizio riga dopo un wrap
                        if not current_tokens:
                            continue
                        if current_w + space_w <= self.width:
                            current_tokens.append(" ")
                            current_w += space_w
                        else:
                            flush_line()
                            # dopo wrap non aggiungiamo spazi iniziali
                    continue

                # Run di testo (senza spazi): può richiedere split charwise
                remaining = run
                while remaining:
                    # Se la riga è piena, vai a capo
                    if current_tokens and current_w >= self.width:
                        flush_line()

                    remaining_w = self.width - current_w
                    run_w = self.font.getlength(remaining)

                    if run_w <= remaining_w:
                        current_tokens.append(remaining)
                        current_w += run_w
                        remaining = ""
                    else:
                        k = self._fit_prefix_len(remaining, remaining_w, self.font.getlength)
                        part = remaining[:k]
                        part_w = self.font.getlength(part)

                        # Safety: se non entra nulla (caso limite), forza 1 char
                        if k <= 0:
                            part = remaining[:1]
                            part_w = self.font.getlength(part)
                            remaining = remaining[1:]
                        else:
                            remaining = remaining[k:]

                        current_tokens.append(part)
                        current_w += part_w
                        flush_line()

            # Chiudi l'ultima linea del paragraph (anche se vuota)
            lines.append(current_tokens)

        return lines


In [16]:
long_text = """
# TOC

Il paper **4D-RGPT (arxiv:2512.17012)** pubblicato da NVIDIA affronta i limiti dei modelli multimodali attuali (MLLM) nella percezione dello spazio 4D (3D + tempo). Sebbene i modelli esistenti eccellano nel vision-language, faticano a comprendere la profondità e la dinamica temporale. 4D-RGPT introduce un framework di **Perceptual 4D Distillation (P4D)** che trasferisce la conoscenza da un modello esperto 4D "frozen" a un LLM student senza aggiungere costi computazionali in fase di inferenza.

L'architettura utilizza un encoder regionale dedicato e un sistema di encoding temporale migliorato per rispondere a domande complesse su oggetti in movimento. Il lavoro introduce anche **R4D-Bench**, un benchmark specifico per il ragionamento 4D a livello di regione, colmando il gap tra la percezione visiva 2D e la comprensione geometrica tridimensionale nel tempo.

- Code: /
- Paper: [https://arxiv.org/pdf/2512.17012](https://arxiv.org/pdf/2512.17012)
- Project Page: [https://ca-joe-yang.github.io/resource/projects/4D_RGPT](https://ca-joe-yang.github.io/resource/projects/4D_RGPT)
- HuggingFace: [https://huggingface.co/papers/2512.17012](https://huggingface.co/papers/2512.17012)

## 4D-RGPT Domande di ricerca

- **Come integrare la percezione 4D a grana fine nei MLLM senza introdurre overhead in fase di inferenza?**
Il paper propone la **Perceptual 4D Distillation (P4D)**. Questa tecnica permette di addestrare lo student MLLM a imitare le rappresentazioni di un encoder esperto 4D. Poiché i moduli di distillazione sono utilizzati solo durante il training e rimossi in fase di produzione, il modello mantiene la velocità di un LLM standard pur possedendo capacità percettive avanzate.
- **In che modo è possibile migliorare la consapevolezza temporale dei token visivi per task di tracking e profondità?**
Gli autori introducono il **Timestamp Positional Encoding (TPE)**. A differenza dei positional encodings standard, il TPE inietta segnali temporali espliciti direttamente nelle feature estratte dall'encoder regionale. Questo permette al trasformatore di distinguere la progressione cronologica degli oggetti all'interno di una sequenza video complessa.
- **Quali sono le lacune dei benchmark attuali nella valutazione del ragionamento 4D?**
I benchmark esistenti si concentrano spesso su scene statiche o descrizioni globali del video. Il paper dimostra che questi non misurano la capacità di "grounding" spaziale. Per questo viene creato **R4D-Bench**, che richiede al modello di identificare regioni specifiche e rispondere a domande sulla loro evoluzione geometrica e interazione nel tempo.

## 4D-RGPT Metodi e Tecniche

L'architettura di 4D-RGPT si basa su tre pilastri fondamentali che permettono la transizione da una comprensione puramente visiva a una percezione spaziale dinamica.

### Perceptual 4D Distillation (P4D)

La tecnica core è la distillazione della percezione 4D. Viene utilizzato un modello insegnante (teacher) specializzato in point cloud o video 4D. Il processo di ottimizzazione minimizza la discrepanza tra le feature dello student e quelle del teacher attraverso una loss combinata:

$$
\mathcal{L}_{distill} = \lambda_{latent} \mathcal{L}_{latent} + \lambda_{explicit} \mathcal{L}_{explicit}
$$

La componente $ \mathcal{L}_{latent} $ agisce nello spazio delle feature latenti, calcolando la distanza tra le mappe di attivazione dell'encoder dello student e del teacher:

$$
\mathcal{L}_{latent} = \sum_{i} \| \phi_{s}(x_i) - \text{proj}(\phi_{t}(x_i)) \|_2^2
$$

### 4D Regional Encoder e TPE

Per gestire il "region-level understanding", il modello utilizza un **4D Regional Encoder**. Questo modulo estrae token specifici per le aree di interesse segnate tramite bounding box. Il **Timestamp Positional Encoding (TPE)** viene aggiunto per preservare l'ordine temporale dei frame:

```python
import torch
import torch.nn as nn

class TimestampPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        self.encoding = nn.Parameter(torch.zeros(max_len, d_model))
        
    def forward(self, regional_features, timestamps):
        # regional_features: [batch, frames, regions, dim]
        # timestamps: indici temporali dei frame
        t_embed = self.encoding[timestamps, :]
        return regional_features + t_embed.unsqueeze(2)
```

### Explicit Distillation Task

Oltre alla loss latente, il modello viene addestrato su task espliciti di ricostruzione 4D, come la predizione del flusso ottico (optical flow) o della profondità relativa. Questo forza lo student a catturare segnali geometrici reali piuttosto che limitarsi a correlazioni testuali.

## 4D-RGPT Dataset

Il contributo principale a livello di dati è **R4D-Bench**. Si tratta di un benchmark per la Region-level 4D Video Question Answering, costruito con una pipeline ibrida automatizzata e verificata da esseri umani. Include scene dinamiche con variazioni di profondità e interazioni tra oggetti.

Per l'addestramento, 4D-RGPT utilizza anche dataset 3D/4D esistenti come **ScanNet** e **Waymo Open Dataset**, riconvertiti per task di istruzione multimodale.

## 4D-RGPT Licenze

Il dataset R4D-Bench e il codice di 4D-RGPT non hanno una licenza esplicitamente dichiarata nel pre-print.

# FAQ - FAQ per 4D-RGPT

## In cosa differisce 4D-RGPT da un normale modello Video-LLM?

La maggior parte dei Video-LLM tratta i video come una sequenza di immagini 2D. 4D-RGPT, invece, "vede" la profondità e la struttura geometrica degli oggetti grazie alla distillazione da modelli esperti di point cloud, permettendo un ragionamento spaziale molto più preciso.

## La Perceptual Distillation rallenta l'inferenza del modello?

No. Uno dei vantaggi chiave è che i moduli del modello "teacher" e le teste di distillazione vengono utilizzate esclusivamente durante la fase di training (training-only). Una volta addestrato, il modello student (4D-RGPT) opera in autonomia senza costi aggiuntivi.

## Quali sono le applicazioni pratiche di questa tecnologia?

4D-RGPT è ideale per ambiti dove la precisione spaziale è critica, come la guida autonoma (per capire la traiettoria di altri veicoli), la robotica collaborativa e l'ispezione industriale assistita da AI, dove è necessario identificare anomalie in oggetti che si muovono nello spazio 3D.

# END_FAQ
""".replace("\n", " ")

In [ ]:
# long_text = "In **Step-DeepResearch Technical Report** (arxiv:2512.20491) il team di StepFun presenta un modello da 32 miliardi di parametri progettato per compiti di \"Deep Research\" (ricerca approfondita a lungo orizzonte). L'approccio sfida il fatto che siano necessari modelli enormi per la ricerca complessa. Il core del lavoro è una strategia di sintesi dei dati basata su **Atomic Capabilities** (capacità atomiche) come pianificazione, raccolta informazioni e scrittura di report. Invece di addestrare il modello su intere traiettorie rumorose, gli autori scompongono il processo in skill fondamentali, addestrate progressivamente tramite un percorso che va dal *mid-training agentico*, al *Supervised Fine-Tuning (SFT)* fino al *Reinforcement Learning (RL)*. Il modello utilizza un'architettura a singolo agente in stile ReAct e viene valutato su un nuovo benchmark, **ADR-Bench**, dimostrando efficienza e qualità paragonabili a modelli closed-source molto più grandi."

gen = ImageTextGen(
    canvas_size=(128, 128), 
    font_path="/Users/mascit/Downloads/CozetteVector.ttf", #"/Users/mascit/Downloads/unifont-17.0.03.otf", #"/Users/mascit/Downloads/CozetteVector.ttf", #"/Users/mascit/Downloads/04b_03/04B_03__.TTF",
    font_size=8,
    custom_space_width=3
)

# Render
result_images = gen.render(long_text)

print(f"Generated {len(result_images)} images.")

# Save for inspection
for i, img in enumerate(result_images):
    img.save(f"output_page_{i}.png")

Generated 11 images.


In [103]:
len(long_text)

964

In [105]:
(128 / 8) **2

256.0

In [54]:
import numpy as np


arr = np.array(img)

In [55]:
np.unique(arr)

array([False,  True])

In [6]:
len(t) / 400 * 90 

1422.45